In [5]:
from dataclasses import dataclass
from functools import partial
from typing import Callable, Optional

import jax
import jax.numpy as jnp
from jax import jit, vmap, grad, random, lax

import os
import pwd
os.chdir("/home/zongchen/thinned_mfld")
from utils.configs import CFG
from utils.problems import Problem
from jaxtyping import Array 
from jax_tqdm import scan_tqdm


In [6]:
def q1_nn(z, x):
    # Simple 2-layer NN for demonstration
    d_hidden = x.shape[0] - 2
    W1, b1, W2 = x[:d_hidden], x[d_hidden+1], x[d_hidden+2]
    h = jnp.tanh(z @ W1 + b1)
    return jnp.dot(W2, h)

In [7]:

Z = jnp.array([[0.5, 1.0], [1.5, -0.5], [-1.0, 2.0]])
d_data = Z.shape[1]
N_particle = 3
x_all = jax.random.normal(jax.random.PRNGKey(0), (N_particle, d_data+2))  # 3 samples of NN params

preds_all = jax.vmap(q1_nn, in_axes=(None, 0))(Z, x_all)
preds = preds_all.mean(axis=0)
print("Predictions:", preds)
print("Recaled Predictions:", preds * N_particle)

Predictions: [0.5570277 0.4093504 0.4733181]
Recaled Predictions: [1.6710831 1.2280512 1.4199543]


In [8]:
def two_layer_nn(x, z):
    """Forward pass: x ∈ R^{d_in}, output ∈ R^{d_out}"""
    W1, b1, W2 = x[:, :d_data], x[:, d_data+1], x[:, d_data+2]
    h = jnp.tanh(z @ W1.T + b1)
    return h @ W2

preds = two_layer_nn(
    x=x_all,
    z=Z,
)
print("Predictions (two_layer_nn):", preds)

Predictions (two_layer_nn): [1.6710831 1.2280512 1.4199543]
